# Compute diversity indices

In [2]:
%run "Header.ipynb"

In [3]:
import pysam
import pysamstats

## Parameters of the diversity index: $p$ and $m$

See paper for description.

In [ ]:
m = 5
# The possible values of p we'll try. These are NOT p-values so i'm not gonna call them that lmao
# We exclude p = 0 since computing m/0 is obviously uh not gonna work out
percentages = [2, 1, 0.5, 0.25, 0.1, 0.05]

## Compute diversity index for varying values of $p$

Note that not all genomes that pass the filter for one value of $p$ will pass the filter for other values of $p$. This is expected.

In [ ]:
# This should include all edges' alignments now, not just the three selected genomes
bf = pysam.AlignmentFile("../main-workflow/output/fully-filtered-and-sorted-aln.bam", "rb")

In [ ]:
p2mincov = {p: m / p for p in percentages}

p2seq2dividx = {p: {} for p in percentages}

for si, seq in enumerate(bf.references, 1):
    pct = 100 * (si / bf.nreferences)
    print(f"On seq {seq} ({si:,} / {bf.nreferences:,}) ({pcf:.2f}%)...")
    
    # TODO: immediately ignore sequences with length < 1 Mbp.
    
    # TODO: Do I need to specify start and end explicitly here, or will the defaults just cover the whole seq?
    # Can get edge length by reading the all_edges.fasta file with skbio I guess.
    # See bam-to-jsons.py for a description of why these params are useful.
    for ri, rec in enumerate(
        pysamstats.stat_variation(
            bf, chrom=seq, fafile="output/all_edges.fasta", truncate=True, pad=True, max_depth=1e6
        ),
        1
    ):
        for p in vals_of_p:
            pass
            # Figure out how many positions have coverage > p2mincov[p]
            # If it exceeds 50% of the length of this sequence, it's sufficiently covered. compute the index and
            # update p2seq2dividx.
            # ah dang I guess we do have to load the fasta file to get sequence length

## Visualize the diversity index

Histograms sharing the x-axis (should go from 0 to 1 I guess -- maybe convert to percentages idk)

Highlight CAMP, BACTERIA, and BACTEROIDALES as vertical lines colored using same colors as logratio / covlen plots on each histogram!